# 5h Model

The process is:
1. Clean up data
    The process is remove Objects features (Categorical Values) that we could convert
    Note. The features were removed since the get_dummy function failed converting values
    We include features based on a Random Forest model (Feature of Importance) 
    Removed Null Values with the MODE values

2. Create Model

    1 - This notebook contains Grid Serach Cross Validation

3. Train
2. Predict (Stats)

In [1]:
import warnings
warnings.simplefilter('ignore')

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
#chunksize returns as a loop through the dataset. 
def transform_data(method, df_chunk):
    chunk_final = pd.DataFrame()
    if method is None:
        for chunk in df_chunk:
            chunk_final = chunk_final.append(chunk)
    else:
        if isinstance(df_chunk, pd.io.parsers.TextFileReader):
            for chunk in df_chunk:
                chunk_mod = method(chunk)
                chunk_final = chunk_final.append(chunk_mod)
        if isinstance(df_chunk, list):
            for chunk in df_chunk:
                chunk_mod = method(chunk)
                chunk_final = chunk_final.append(chunk_mod)
    return chunk_final

In [3]:
def cleanup_data(data_frame):
    print("Start cleanup_data")
    for column in data.columns:
        if data_frame[column].isnull().sum() > 0:
            print(column)
            data_frame[column].fillna(data_frame[column].mode().iloc[0], inplace=True)
    #data_frame = data_frame.fillna(data_frame.mean())
    #data_frame = data_frame.fillna(0)
    return data_frame

In [4]:
#def get_X(dataframe):
#    print("start get_X")
#    X = dataframe.drop("HasDetections", axis=1)
#    X = pd.get_dummies(X, 
#                   columns=["MachineIdentifier", "ProductName", "EngineVersion", "AppVersion", "AvSigVersion", "Platform", "Processor", "OsVer", "OsPlatformSubRelease", "OsBuildLab", 
#                            "SkuEdition", "PuaMode", "SmartScreen", "Census_MDC2FormFactor", "Census_DeviceFamily", "Census_ProcessorClass", "Census_PrimaryDiskTypeName", "Census_ChassisTypeName", "Census_PowerPlatformRoleName", 
#                            "Census_InternalBatteryType", "Census_OSVersion", "Census_OSArchitecture", "Census_OSBranch", "Census_OSEdition", "Census_OSSkuName", "Census_OSInstallTypeName", 
#                            "Census_OSWUAutoUpdateOptionsName", "Census_GenuineStateName", "Census_ActivationChannel", "Census_FlightRing"], drop_first=True)
#    return X

def get_X(dataframe): 
    print("start get_X")
    print(dataframe.shape)
    dataframe = dataframe.drop(["MachineIdentifier", "HasDetections", "AvSigVersion", "OsBuildLab", 
                       "Census_MDC2FormFactor", "Census_DeviceFamily", "Census_ProcessorClass", "Census_PrimaryDiskTypeName", "Census_ChassisTypeName", 
                        "Census_PowerPlatformRoleName", "Census_InternalBatteryType", "Census_OSVersion", "Census_OSArchitecture", "Census_OSBranch", "Census_OSEdition", "Census_OSSkuName", 
                        "Census_OSInstallTypeName", "Census_OSWUAutoUpdateOptionsName", "Census_GenuineStateName", "Census_ActivationChannel", "Census_FlightRing"]
                        , axis=1)
    #"MachineIdentifier", "AvSigVersion", "OsBuildLab", 
    #"Census_MDC2FormFactor", "Census_DeviceFamily", "Census_ProcessorClass", "Census_PrimaryDiskTypeName", "Census_ChassisTypeName", 
    #"Census_PowerPlatformRoleName", "Census_InternalBatteryType", "Census_OSVersion", "Census_OSArchitecture", "Census_OSBranch", "Census_OSEdition", "Census_OSSkuName", 
    #"Census_OSInstallTypeName", "Census_OSWUAutoUpdateOptionsName", "Census_GenuineStateName", "Census_ActivationChannel", "Census_FlightRing"
            
    columns=["ProductName", "EngineVersion", "AppVersion", "Platform", "Processor", "OsVer", "OsPlatformSubRelease", "SkuEdition", "PuaMode", "SmartScreen"]
    
    for column in columns:
        print(column)
        print(dataframe.shape)
        dataframe = pd.get_dummies(dataframe, columns=[column], drop_first=True)
        print(dataframe.shape)
    
    return dataframe

In [18]:
def get_fields(dataframe): 
    stats = pd.DataFrame(columns=["column", "type", "total"])
    dataframeI = dataframe
    dataframeI = dataframeI.drop(["MachineIdentifier", "HasDetections", "AvSigVersion", "OsBuildLab", 
                       "Census_MDC2FormFactor", "Census_DeviceFamily", "Census_ProcessorClass", "Census_PrimaryDiskTypeName", "Census_ChassisTypeName", 
                        "Census_PowerPlatformRoleName", "Census_InternalBatteryType", "Census_OSVersion", "Census_OSArchitecture", "Census_OSBranch", "Census_OSEdition", "Census_OSSkuName", 
                        "Census_OSInstallTypeName", "Census_OSWUAutoUpdateOptionsName", "Census_GenuineStateName", "Census_ActivationChannel", "Census_FlightRing"]
                        , axis=1)
    
    for column in dataframeI.columns:
        #print(column)
        #print(data[column].dtype)
        #print(len(data[column].unique()))
        stats = stats.append({ "column" : column, 
                      "type" : dataframeI[column].dtype, 
                      "total" : len(dataframeI[column].unique())}, ignore_index=True)

    print(len(stats))
    stats.to_csv("features_count.csv", index_col = False)

In [5]:
#df_chunk = pd.read_csv('Resources/train.csv', chunksize=500000)
data = pd.read_csv('Resources/train.csv.gz', compression="gzip")
#data = transform_data(None, df_chunk)
print(data.shape)

(8921483, 83)


In [6]:
y = data[["HasDetections"]]
print(y.shape)

(8921483, 1)


In [7]:
data = get_X(data)
data.head()

start get_X
(8921483, 83)
ProductName
(8921483, 62)
(8921483, 66)
EngineVersion
(8921483, 66)
(8921483, 134)
AppVersion
(8921483, 134)
(8921483, 242)
Platform
(8921483, 242)
(8921483, 244)
Processor
(8921483, 244)
(8921483, 245)
OsVer
(8921483, 245)
(8921483, 301)
OsPlatformSubRelease
(8921483, 301)
(8921483, 308)
SkuEdition
(8921483, 308)
(8921483, 314)
PuaMode
(8921483, 314)
(8921483, 314)
SmartScreen
(8921483, 314)
(8921483, 333)


,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,...,SmartScreen_Prompt,SmartScreen_Promt,SmartScreen_RequireAdmin,SmartScreen_Warn,SmartScreen_off,SmartScreen_on,SmartScreen_prompt,SmartScreen_requireAdmin,SmartScreen_requireadmin,SmartScreen_warn
0,0,7.0,0,NaN,53447.0,1.0,1.0,1,29,128035.0,...,0,0,0,0,0,0,0,0,0,0
1,0,7.0,0,NaN,53447.0,1.0,1.0,1,93,1482.0,...,0,0,0,0,0,0,0,0,0,0
2,0,7.0,0,NaN,53447.0,1.0,1.0,1,86,153579.0,...,0,0,1,0,0,0,0,0,0,0
3,0,7.0,0,NaN,53447.0,1.0,1.0,1,88,20710.0,...,0,0,0,0,0,0,0,0,0,0
4,0,7.0,0,NaN,53447.0,1.0,1.0,1,18,37376.0,...,0,0,1,0,0,0,0,0,0,0


In [8]:
data = cleanup_data(data)

Start cleanup_data
RtpStateBitfield
DefaultBrowsersIdentifier
AVProductStatesIdentifier
AVProductsInstalled
AVProductsEnabled
CityIdentifier
OrganizationIdentifier
GeoNameIdentifier
IsProtected
SMode
IeVerIdentifier
Firewall
UacLuaenable
Census_OEMNameIdentifier
Census_OEMModelIdentifier
Census_ProcessorCoreCount
Census_ProcessorManufacturerIdentifier
Census_ProcessorModelIdentifier
Census_PrimaryDiskTotalCapacity
Census_SystemVolumeTotalCapacity
Census_TotalPhysicalRAM
Census_InternalPrimaryDiagonalDisplaySizeInInches
Census_InternalPrimaryDisplayResolutionHorizontal
Census_InternalPrimaryDisplayResolutionVertical
Census_InternalBatteryNumberOfCharges
Census_OSInstallLanguageIdentifier
Census_IsFlightingInternal
Census_IsFlightsDisabled
Census_ThresholdOptIn
Census_FirmwareManufacturerIdentifier
Census_FirmwareVersionIdentifier
Census_IsWIMBootEnabled
Census_IsVirtualDevice
Census_IsAlwaysOnAlwaysConnectedCapable
Wdft_IsGamer
Wdft_RegionIdentifier


In [9]:
print(data.shape, y.shape)

(8921483, 333) (8921483, 1)


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, random_state=1, stratify=y)
print(X_train.shape, y_train.shape)

(6691112, 333) (6691112, 1)


In [11]:
print(X_train.shape, y_train.shape)

(6691112, 333) (6691112, 1)


In [12]:
# Create the SVC pipeline
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC 
model_svc = make_pipeline(SVC(kernel='linear'))
model_svc

Pipeline(memory=None,
     steps=[('svc', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [13]:

# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model_svc, param_grid, verbose=3)

In [ ]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] svc__C=1, svc__gamma=0.0001 .....................................


In [ ]:
print(grid.best_params_)

In [ ]:
print(grid.best_score_)

In [ ]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)

In [ ]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["blue", "red"]))